以下の設定になっていること  

ランタイム > ランタイムのタイプを変更  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

編集 > ノートブックの設定  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

In [2]:
# googleドライブをマウント
from google.colab import drive 
drive.mount('/content/drive')
# 表示されるリンクをクリックして、アクセスを許可して、最後に表示される文字列を以下の入力欄に入れる

Mounted at /content/drive


In [3]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [4]:
# importで使う必要があるので、インストールがランタイム切れるごとに必要
# インストール後にランタイムの再起動を行わないとT5Tokenizerが見つからない
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# ドライブに保存してるものでインストール
!pip install -e transformers

# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

Obtaining file:///content/drive/My%20Drive/work/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3 MB 9.3 MB/s 
     |████████████████████████████████| 895 kB 74.2 MB/s 
  Running setup.py develop for transformers
     |████████████████████████████████| 159 kB 8.4 MB/s 
     |████████████████████████████████| 243 kB 64.7 MB/s 
     |████████████████████████████████| 69 kB 8.9 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0
     |████████████████████████████████| 1.1 MB 9.0 MB/s 


In [1]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


CLM（Causal Language Modeling）: GPT、GPT-2

・model_name_or_path: モデルのチェックポイント（モデルを最初から学習しない場合）  
・model_type: モデルの種別（モデルを最初から学習する場合）  
・config_name: コンフィグ名（model_nameと同じでない場合）  
・tokenizer_name: トークナイザー名（model_nameと同じでない場合）  
・cache_dir: キャッシュフォルダ  
・use_fast_tokenizer: Fastトークナイザーを使用するかどうか  
・model_revision: 使用するモデルの特定のバージョン  
・use_auth_token: 「transformers-cli login」の実行時に生成されたトークンを使用するかどうか  

・dataset_name: データセット名  
・dataset_config_name: データセットのコンフィグ名  
・train_file: 学習データ（テキストファイル）  
・validation_file: 検証データ（テキストファイル）  
・overwrite_cache: キャッシュの上書き  
・validation_split_percentage: 学習データから使われる検証データの割合（検証データがない場合）  
・max_seq_length: トークン化後の最大合計入力シーケンス長  
・preprocessing_num_workers: 前処理に使用するプロセス数  
・block_size: トークン化後のオプションの入力シーケンス長  
・max_train_samples: 学習データの最大数  
・max_val_samples: 検証データの最大数  

    # GPT2のモデルファイルを指定
    --model_name_or_path=rinna/japanese-gpt2-medium \
    # 学習ファイル
    --train_file=train.txt \
    # 評価データファイル
    --validation_file=train.txt \
    # トレーニングを実施する
    --do_train \
    # 評価を実施する
    --do_eval \
    # 学習回数（エポック数）
    --num_train_epochs=30 \
    # チェックポイントの保存間隔
    --save_steps=5000 \
    # チェックポイントの保持数
    --save_total_limit=3 \
    # T5Tokenizer.model_max_length=1024をチャンクサイズとして使用するかblock_sizeで指定するかを設定する（設定しないとtokenizerの超大なサイズから1024になる）メモリに乗るように調整する必要がある（バッチサイズとの兼ね合い）
    --block_size=512 \
    # GPU1つあたりの学習バッチサイズ
    --per_device_train_batch_size=2 \
    # GPU1つあたりの評価バッチサイズ
    --per_device_eval_batch_size=2 \
    # モデルとチェックポイントの出力先
    --output_dir=output/ \
    # 出力先の上書きの許可
    --overwrite_output_dir=True \
    # T5Tokenizerで高速化ライブラリがあれば使用する
    --use_fast_tokenizer=False

### 学習する回数
エポック数 * (データ文字数 / ブロックサイズ):1つの学習データ / バッチサイズ  
    エポック数 = 1  
    データ文字数 = 22142  
    データの文字数だと数字が合わないので、tokenizeした結果の形態素数による？
    ブロックサイズ = 512  
    データ文字数 / ブロックサイズ = 43  
    バッチサイズ = 1  
    Total optimization steps = 29  


バッチサイズ = 2  
バッチサイズを2にすることで、トータルの実行回数が減っている  
Total optimization steps = 15  

In [ ]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --do_train \
    --num_train_epochs=300 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --block_size=256 \
    --per_device_train_batch_size=4 \
    --output_dir=output/ \
    --overwrite_output_dir=True \
    --use_fast_tokenizer=False

2021-07-28 12:28:23.411625: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
07/28/2021 12:28:32 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
07/28/2021 12:28:32 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=output/, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=300.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Jul28_12-28-31_69031f40117e, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, l

In [ ]:
# from transformers import T5Tokenizer, AutoModelForCausalLM

# # トークナイザーとモデルの準備
# tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
# model = AutoModelForCausalLM.from_pretrained("output/")

In [8]:
# もちろんだが、Autoでも直指定でも同じ結果にはなっている
# https://huggingface.co/transformers/model_doc/gpt2.html#gpt2lmheadmodel
from transformers import T5Tokenizer, GPT2LMHeadModel

# GPU判定
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = GPT2LMHeadModel.from_pretrained("output/")
model.to(device)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(32000, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2):

In [ ]:
# 推論
# https://huggingface.co/blog/how-to-generate
# https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate
input = tokenizer.encode("左のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブ 残り10秒!
左のジャブ 1ラウンド10左のフック!
左のジャブ 1ラウンド2r
左のジャブ 残り2分間という戦いになったが、しっかりお互いがパンチを出し合う中での戦いになった。
左のジャブ 残り2分間、そのボクサー人生にとって、これ以上ないような勝利をつかみたいと話していた。
左のジャブ 残り2分間という戦いになりました。
左のジャブ このストレート!
左のジャブ 残り2分間という戦いになりました。
左のジャブ 残り 2分間という戦いになるが、しっかりガードして右のストレート!
左のジャブ 次は右のストレート右のフック!右のストレートも返していく!


In [ ]:
input = tokenizer.encode("左のフック", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のフック 今回はを狙いたいと話していましたが、気持ちのぶつかり合いだ。
左のフック このストレートどうでしょうか?
左のフック これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。
左のフック これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。
左のフック これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。
左のフック 今回はを狙いたいと話していましたが、気持ちのぶつかり合いだ。
左のフック これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。
左のフック これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。
左のフック 今回はを狙いたいと話していましたが、気持ちのぶつかり合いだ。
左のフック これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。


In [ ]:
input = tokenizer.encode("ボディ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボディ このダウンを奪って一気に行こうかというところから、もう一度ギアを落としました。
ボディ このダウンフォース、かなり頑張ってますね。
ボディ 得意の右アッパー!
ボディ 選手は152cmで体重が46キロですから、およそ5倍の重りがかかっているということになります。
ボディ 選手は3回戦で行われます。
ボディ このダウンを奪って一気に行こうかというところから、もう一度ギアを落としました。
ボディ ただ、選手はそれだけではありませんでした。
ボディ このストレートのまいは、まさにのですね。
ボディ 選手は25歳これがキャリア10戦目。
ボディ これが4戦目。


In [ ]:
input = tokenizer.encode("右のストレート", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のストレート さあ今度はの右。
右のストレート 少し行き過ぎになったか。
右のストレート ここはガードを固めたほうが良いかなと。
右のストレート いまパンチをもらっているです。
右のストレート 少しが下がったように見えました。
右のストレート ここはガードを固めたガードを固めた左のジャブ。
右のストレート さあ第2ラウンドが始まりました。
右のストレート 少しクラっときたか。
右のストレート さん、ボディがいいんで、ボディからのフックの返しが、さんの素早いスピードの打ち終わりを狙ってるイメージで、いいじゃないですか。
右のストレート 少しクラっときたか。


In [ ]:
input = tokenizer.encode("ガードの上から", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=10)
result = tokenizer.batch_decode(output, skip_special_tokens=True)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上から がを打ちました。
ガードの上から この距離から両者引きませんね。
ガードの上から この距離で両者引きませんね。
ガードの上から この距離で両者引きませんね。
ガードの上から この距離で両者全く引きませんね。
ガードの上から これよりいよいよ今日の試合のメインイベントお届けします
ガードの上から この距離で両者引きませんね。
ガードの上から でもしっかり手数を出して。
ガードの上から 今回はを狙いたいと話していましたが、気持ちのぶつかり合いだ。
ガードの上から これはの距離か?


tokenizerの中身を確認  
\<s\>の意味合いを表示  


In [ ]:
# model.generateの結果はtokenizerのindexベクトル
output[4]

tensor([    9,  5682,    10, 12276,     2,     9,     0,    20,  2115,    18,
         5456,  2199,     7,    80,     0,    10,   819,     8,     2,     2,
            2,     2,     2])

In [ ]:
# 記号の意味
tokenizer.all_special_tokens

['<s>', '</s>', '<unk>', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [ ]:
# 記号に対応するindex
tokenizer.all_special_ids

[1, 2, 0, 5, 3, 4, 6]

{"bos_token": "\<s\>", "eos_token": "\</s\>", "unk_token": "<unk>", "sep_token": "[SEP]", "pad_token": "[PAD]", "cls_token": "[CLS]", "mask_token": "[MASK]"}  
bos_token: 文の先頭（Begin of sequence token）  
eos_token: 文のおしり（End of Sequence token）  
unk_token: IDに変換できない文字（Unknown token）  
sep_token: 文と文を区切り目（The separator token）  
pad_token: パッディング（The token used for padding）  
cls_token: 分類用（cls_token）  
mask_token: マスク（The token used for masking values）  

* https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.batch_decode  
sequences : torch.Tensorの配列を入力値として指定  
  トークン化された入力IDのリスト  
skip_special_tokens : デコード時に特殊なトークンを削除するかどうか(eos_tokenとかを消す)(デフォルト:False)  
clean_up_tokenization_spaces : トークン化スペースをクリーンアップするかどうか(デフォルト:True)  

* https://huggingface.co/transformers/internal/tokenization_utils.html#transformers.tokenization_utils_base.PreTrainedTokenizerBase.encode  

text (str, List[str] or List[int]) – 入力文字列  

text_pair (str, List[str] or List[int], optional) – ペアとなるもう一つを入力する場合のオプション  

add_special_tokens (bool, optional, defaults to True) – 上記で定義していない特別なトークンをモデルに適用するか

padding (bool, str or PaddingStrategy, optional, defaults to False) –パディングして入力シーケンスを揃える場合  

truncation (bool, str or TruncationStrategy, optional, defaults to False) –逆に長過ぎる場合に、一定の長さに揃える場合

max_length (int, optional) –トランケーション・パディングで使用するオプション

stride (int, optional, defaults to 0) – max_lengthで切り捨てられたのを調整する  

is_split_into_words (bool, optional, defaults to False) – 単語分割が既にされている場合True

pad_to_multiple_of (int, optional) – 指定された値の倍数になるようにシーケンスをパッドする  

return_tensors (str or TensorType, optional) – python整数のリストの代わりにテンソルを返す  
'tf': Return TensorFlow tf.constant objects.  
'pt': Return PyTorch torch.Tensor objects.  
'np': Return Numpy np.ndarray objects.  

* https://huggingface.co/blog/how-to-generate  
* https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.generation_utils.GenerationMixin.generate  
* https://note.com/npaka/n/n5d296d8ae26d  
* https://note.com/npaka/n/n96dde45fdf8d  

### GPT2LMHeadModel.generateのオプションを確認  

input_ids (torch.LongTensor of shape (batch_size, sequence_length), optional) – 入力シーケンス  

max_length (int, optional, defaults to model.config.max_length) – 生成されるシーケンスの最大長を指定（学習に使用した256の長さが良さそう）

max_new_tokens (int, optional, defaults to None) – 現在のトークン数に関係なく、生成されるシーケンスの最大長を指定

min_length (int, optional, defaults to 10) – 生成されるシーケンスの最小の長さ

do_sample (bool, optional, defaults to False) – 単語予測にサンプリングを入れてランダム性を導入する。（デフォルトは greedy decoding の生成）  
https://zenn.dev/hellorusk/articles/1c0bef15057b1d  

early_stopping (bool, optional, defaults to False) – ビーム探索で、num_beams個の文が生成された時点で、ビーム探索を終了するかどうか  

num_beams (int, optional, defaults to 1) – ビームサーチを行うビームの数。1はビームサーチを行わないことを意味します。  

temperature (float, optional, defaults to 1.0) – 次のトークンの確率をモジュール化するために使用される値です。温度（デフォルト1、推奨0.7〜1.0）ボルツマン分布のパラメータ。小さい値ではランダムな補完が減り，0では決まりきった繰り返しの文になる。大きい値ではより様々な補完がされる。  

top_k (int, optional, defaults to 50) – top-k-filteringのために保持する最高確率の語彙トークンの数です。確率が大きめな候補からサンプリングしてランダム性を導入する際の候補を何個にするか。40が一般的に良い値  

top_p (float, optional, defaults to 1.0) – 生成テキストを累積確率に制限 (0で制限なし) float < 1に設定すると、top_p以上の確率を持つ最も確率の高いトークンのみが生成のために保持されます。  
https://zenn.dev/hellorusk/articles/1c0bef15057b1d#top-p-(nucleus)-sampling  

repetition_penalty (float, optional, defaults to 1.0) – 反復ペナルティのパラメータです。1.0はペナルティなし。すでに生成された単語や文脈に属する単語にペナルティを与えるために使用することができます。反復防止にはかなり効果的ですが、異なるモデルやユースケースには非常に敏感なようで、議論がある。  

pad_token_id (int, optional) – PADトークンを指定

bos_token_id (int, optional) – bosトークンを指定

eos_token_id (int, optional) – eosトークンを指定

length_penalty (float, optional, defaults to 1.0) – 長さに対する指数関数的なペナルティ。1.0はペナルティがないことを意味します。1.0未満の値を設定すると、モデルは短い配列を生成するようになり、1.0以上の値を設定すると、モデルは長い配列を生成するようになります。

no_repeat_ngram_size (int, optional, defaults to 0) – int > 0に設定すると、そのサイズのngramはすべて一度しか発生しません。最も一般的な n-grams ペナルティは、すでに見た n-gramsを作る可能性のある次の単語の確率を 0 に手動で設定することで、n-gramsが 2 回出現しないようにするものです。  

encoder_no_repeat_ngram_size (int, optional, defaults to 0) – int > 0に設定すると、encoder_input_idsに出現したそのサイズのすべてのngramは、decoder_input_idsには出現しません。

bad_words_ids (List[List[int]], optional) – 生成してはいけないトークンのidのリスト。トークンのIDは以下で確認  
tokenizer(bad_word, add_prefix_space=True).input_ids  

num_return_sequences (int, optional, defaults to 1) – バッチ内の各要素について、独立して計算された戻り値の配列の数(返却される結果の数)。返されるべき最高得点のBeamの数を設定します。ただし、num_return_sequences <= num_beams とします。

max_time (float, optional, defaults to None) – 計算の実行を許可する最大時間を秒単位で指定します。割り当てられた時間が経過しても、生成は現在のパスを終了します。  

attention_mask (torch.LongTensor of shape (batch_size, sequence_length), optional) – パディングされたトークンのインデックスに対してアテンションを行わないようにするためのマスクです。マスクの値は [0, 1] で、マスクされていないトークンには 1、マスクされたトークンには 0 です。提供されていない場合は、パッドトークンをマスクするinput_idsと同じ形のテンソルがデフォルトになります。attentionで予測するための配列を作るので、マスクすると候補に出なくなる。入力シーケンスに対して同じ長さで指定する

decoder_start_token_id (int, optional) – エンコーダ・デコーダモデルがbosとは異なるトークンでデコードを開始した場合、そのトークンのid。

use_cache – (bool, optional, defaults to True): 過去の最後のキー／バリューの注目度（モデルに該当する場合）を利用して、デコーディングを高速化するかどうか。  

num_beam_groups (int, optional, defaults to 1) – num_beamsを分割するグループの数（ビームの異なるグループ間の多様性を確保するため）。

diversity_penalty (float, optional, defaults to 0.0) – この値は、ある時点で他のグループのビームと同じトークンを生成した場合、ビームのスコアから差し引かれます。なお、ダイバーシティペナルティは、グループビーム検索が有効な場合にのみ有効です。  

prefix_allowed_tokens_fn – (Callable[[int, torch.Tensor], List[int]], optional):提供された場合、この関数は、各ステップで許可されたトークンのみにビーム検索を制約します。提供されない場合、制約は適用されません。この関数は2つの引数をとります：バッチID batch_id と input_id です。これは、バッチID batch_idと以前に生成されたトークンinput_idsを条件として、次の生成ステップで許可されたトークンのリストを返さなければなりません。この引数は、「自己回帰的実体検索」で説明されているように、接頭辞を条件とした制約付き生成に役立ちます。  

output_attentions (bool, optional, defaults to False) – すべてのアテンションレイヤーのアテンションテンソルを返すかどうか。  

output_hidden_states (bool, optional, defaults to False) – すべてのレイヤーの隠れた状態を返すかどうか。  

output_scores (bool, optional, defaults to False) – 予測スコアを返すかどうか。  

return_dict_in_generate (bool, optional, defaults to False) – 単なるタプルではなく、ModelOutputを返すかどうか。  

forced_bos_token_id (int, optional) – decoder_start_token_idの後に、最初に生成されるトークンとして強制的に使用するトークンのidです。mBARTのような多言語モデルで、最初に生成されるトークンがターゲット言語のトークンである必要がある場合に便利です。（一番最初に生成される単語を指定してしまう。）  

forced_eos_token_id (int, optional) – max_lengthに達したときに、最後に生成されたトークンとして強制的に使用するトークンのidです。(最後をわかりやすくして、途中で切られたのを知らせる)  

remove_invalid_values (bool, optional) – 生成方法がクラッシュするのを防ぐために、モデルの可能性のあるnanとinfの出力を削除するかどうか。remove_invalid_valuesを使うと生成が遅くなることに注意してください。  

synced_gpus (bool, optional, defaults to False) – max_lengthまでwhileループを続けて実行するかどうか  

最新の研究により、単純な Beam Search や Greedy Search が同じ単語列の繰り返しを発生させてしまうのは、decoding に問題があるのではなくモデルの学習自体に問題があるとされています。また、Top-K や Top-p のようなサンプリングによる decoding であってもそうした単語列の繰り返しは発生しうるそうです。  

In [ ]:
input = tokenizer.encode("左のフック", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のフック</s> これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。 少しリズムは良くってきた。 左のボディです!</s>
左のフック</s> これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。 少しリズムは良くってきた。 左のボディです!</s>
左のフック</s> この距離で左のストレート! <unk> の右! 距離が詰まる! 右のストレート! <unk> がストレートを打ち返していく!</s>
左のフック</s> これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。 少しリズムは良くってきた。 左のボディです!</s>
左のフック</s> 今回は<unk> を狙いたいと話していましたが、気持ちのぶつかり合いだ。 距離を詰めてお互い打ち合う!</s> </s> </s>
左のフック</s> これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。 少しリズムは良くってきた。 左のボディです!</s>
左のフック</s> これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。 少しリズムは良くってきた。 左のボディです!</s>
左のフック</s> 今回は<unk> を狙いたいと話していましたが、気持ちのぶつかり合いだ。 距離を詰めてお互い打ち合う!</s> </s> </s>
左のフック</s> 今回は<unk> を狙いたいと話していましたが、気持ちのぶつかり合いだ。 距離を詰めてお互い打ち合う!</s> </s> </s>
左のフック</s> これがもっと距離が近くなると当たるというところなんですが、なかなか捉えられません。 少しリズムは良くってきた。 左のボディです!</s>


In [3]:
input = tokenizer.encode("ラッシュラッシュ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ラッシュラッシュ</s> このラッシュのシーンですけれども、ちょっとよろけたところで、左に右パンツあって行きます。そして、右のストレートも出していきます。</s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> しかし、<unk> も再び世界に向けての挑戦となっていますから、ここで負けてはいけませんよね。 <unk> 選手にとってはここでの負けは厳しいですね。</s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> しかし<unk> も今あっと言わせるシーン作りましたね。いいラッシュましたね。今のはもう狙ってたと見ていいですか?</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> <unk> 選手がスタミナが切れてきたのか、体が流れ始めてきているので、そうなってくると、<unk> 選手のペースになっていきそうな気がしますね。</s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> 会場からは大きな拍手、女子フライ級4回戦ファイナルラウンドへと入っていきます。 左のジャブ、残り2分間という戦いになります。</s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> このラッシュのシーンですけれども、ちょっと後ろに二歩三歩下がったりも見えますけれども。 足の位置も悪くなっているので。</s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> しかし<unk> はそれを許さない。今度は<unk> の右ボディ! ラッシュ打っているときにボディ打たれると効くと思います。 右!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ラッシュラッシュ</s> このラッシュのシーンですけれども、少し後ろに二歩三歩下がったりも見えますけれども。 これは足の位置が上がって、パンチが当たるというのも見えますね。</s>
ラッシュラッシュ</s> <unk> 選手がスタミナが切れてきたのか、体が流れ始めてきているので、そうなってくると、<unk>

In [4]:
input = tokenizer.encode("ダウン", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ダウン</s> ここまでは落ち着いた試合展開。第4ラウンドに入っていきます。 第4ラウンドに入っていきます。 <unk> Cの速いこの左のジャブ、左のジャブからの右、そして右のジャブ、右のストレートを繰り出していったのが<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> <

In [5]:
input = tokenizer.encode("あーとここで倒れた", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 右のフック!左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s>
あーとここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> <

In [6]:
input = tokenizer.encode("ここで倒れた", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.85, temperature=0.7, min_length=64, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ここで倒れた</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。 そうですね。足の位置も悪くなっているので。 ショートレンジのパンチ、ボディに打ち込んでいく青森。 青森がムキになった時左下にお客さん入ってしまったので戻りになってしまったからですね</s> </s> </s> </s> </s>
ここで倒れた</s> 少しクラっときたか。足が止まっている。<unk> がそこに合わせて打っていく。ガードが下がっている。 <unk> がプレスかけ始めましたね。 少しパンチをもらったが、返して行きます<unk> です。ガードが下がってきた。 <unk> がプレスかけ始めましたね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ここで倒れた</s> 少し聞いただけではどのような試合になるか全く想像がつきませんね。 第1ラウンド、左のジャブを打って行ったのは<unk> です。 24歳、ベテランの選手です。 <unk> がどう入ってくるかの点もポイントですね 。 キャリア49戦目、35歳の<unk> 、負ければもう引退も覚悟しなくてはならないという、そんなことも話していました。</s>
ここで倒れた</s> 少し聞いただけでも、そのダメージはすごく伝わってきます。 少し聞いただけでも、そのダメージはすごく伝わってきます。 第1ラウンド、ダウンを食らった直後のラウンドは、ちょっと今までの戦い方に疑問を抱きながらという風に見えましたけれども。 今もうちょっと開き直ってますね。いいと思います今のままで。足も動いてると思うんで。</s>
ここで倒れた</s> 少しふらついた!そしてパンチを貰った!さあ今度は<unk> が返していく! <unk> 選手もいいボディを打ちましたね。 この試合は後半に入ります。 第4ラウンドに入りました。 <unk> Cの速いこの左のジャブ、左のジャブからの右、そして右のジャブ、右のストレートを繰り出していったのが<unk> です。</s> </s> </s> </s>
ここで倒れた</s> 少しクラっときたか。。。 そして少しずつ距離を詰めて

In [7]:
input = tokenizer.encode("ガードの上から連打", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上から連打</s> 少しパンチを貰ったか<unk> が下がりました。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </

In [10]:
input = tokenizer.encode("倒れたノックアウトです。", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 少しクラっときたか。 <unk> が来るところにジャブを入れるだけで止まりはするので。 残りは10秒切りました。</s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート!こちらもしっかり手数を出して。</s> </s>
倒れたノックアウトです。</s> 一気に試合がうごくというところ、左に右パンツあって行きます。 左のストレート! 残り1分。</s> </s> </s>


In [11]:
input = tokenizer.encode("倒れたKOです。", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


倒れた<unk> です。</s> 少しペースが落ちましたが、最後までしっかり食べました。<unk> 選手もいいボディを打ちましたね。 終盤の戦いになりましたが、最後までしっかり<unk> はパンチを打ちました。</s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 少し落ち着いたように見えますね。<unk> 選手。 少しずつ距離が縮まってきて、<unk> のパンチが当たるというのも見えてきました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 少し落ち着いた様子にも見えますが、<unk> には気をつけて欲しいですね。 終盤の戦いになってきます第7ラウンド</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 少し動きが止まってきたか。。 再び世界に向けての挑戦となっています。 56キロ契約8回戦で行われます</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 22歳、キャリア10戦目。もうすでに後はどう終わるかだということを話しています。 22歳の若武者相手にキャリアの多さ、引き出しの多さ、今日そこまでは存分に見せてきました。</s>
倒れた<unk> です。</s> 22歳、キャリア10戦目。もうすでに後はどう終わるかだということを話しています。 右のジャブ。そして入ってきたところに左を合わせるといったパンチですかね。</s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 22歳、ベテランと若手! <unk> の右。 キャリア10戦目。もうすでに後はどう終わるかだということを話しています。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
倒れた<unk> です。</s> 少しパンチが当たりましたでしょうか

In [3]:
input = tokenizer.encode("右のフックから左右連打", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のフックから左右連打</s> ちょっとクラっときたか。<unk> の右、<unk> の右、左右、左右振り分けて行きます。</s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> 左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のフックから左右連打</s> 左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のフックから左右連打</s> 左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のフックから左右連打</s> 左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のフックから左右連打</s> ちょっとクラっときたか。<unk> の右、<unk> の右、左右、左右振り分けて行きます。</s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> 少し<unk> が下がったように見えました。 左のストレート! <unk> が下がった! 左のボディ!</s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> 左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のフックから左右連打</s> ちょっとクラっときたか。<unk> の右、<unk> の右、左右、左右振り分けて行きます。</s> </s> </s> </s> </s> </s> </s> </s> </s>
右のフックから左右連打</s> 左のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>


In [4]:
input = tokenizer.encode("ガードの上からボディー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ガードの上からボディー</s> この距離から両者引きませんね。 右!右のジャブ。右!今<unk> 狙ってましたか。 いいジャブが出ていますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> この距離から両者引きませんね。 右!右のジャブ。右のストレート右のフック!右!今<unk> 狙ってましたか!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> この距離から両者引きませんね。 右!右のジャブ。右のストレート!右!右のジャブ!右のストレート顔面をとらえた強いパンチ。右が非常に伸びている。右のストレートです。</s>
ガードの上からボディー</s> この距離から両者引きませんね。 右!右のジャブ。右のストレート右のフック!右!今<unk> 狙ってましたか!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> この距離から両者引きませんね。 右のジャブ。左のフック。右!右のストレート右!右のフック!引かない!引かせません!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> この距離から両者引きませんね。 右!右のジャブ。右!右のジャブ!今<unk> は右を返して行きます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ガードの上からボディー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。 第7ラウンドも残り1分を切っています。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>


In [3]:
input = tokenizer.encode("ジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ジャブ</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> 右のストレート!右のストレート顔面をとらえた強いパンチ!右が非常に伸びている。右のストレートです! 今パンチを貰ってますが、少し<unk> が下がったように見えます。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> <unk> ・<unk> が このラウンド 残り10秒となりました。 左のストレート! <unk> 選手は中盤にかけて<unk> 勝利、このコロナ禍での応援に来てくださる方に対して、面白い試合をしたいと、言っていますが。</s>
ジャブ</s> さあ、この辺りの入りですけれども、<unk> さん、どんなとこに注目されてますか? 前回は<unk> さんの巻き上げに冷静に対処した<unk> さんましたが、今回はどうでしょうか?</s> </s> </s> </s> </s>
ジャブ</s> 右のストレート右のフック右のフック<unk> さん 残り10秒 左のストレート!右のストレート<unk> も返していく!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> 右のストレート!左のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ジャブ</s> さあ、第6ラウンドに入っています。 左のストレート! <unk> 選手は中盤にかけて<unk> 勝利、

In [4]:
input = tokenizer.encode("左のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブ</s> このストレートどうでしょうか? このストレートどうでしょうか?右のフック、アッパー! 今のこの体のフリというかリズムの感じを見てると、<unk> 選手はしっかり落ち着いて試合展開ができているなと思いました。</s> </s> </s> </s>
左のジャブ</s> 残り2分間という戦いになりました。 第3ラウンド、右のストレート! <unk> 選手は中盤にかけて<unk> 勝利、このコロナ禍での応援に来てくださる方に対して、面白い試合をしたいと、言っていますが。</s>
左のジャブ</s> 残り2分間という戦いになりました。 第7ラウンドに試合が入りました。 <unk> 、左のジャブ、残り2分間という戦いになりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> 今回は<unk> を狙いたいと話していましたが、気持ちのぶつかり合いだ。 距離を詰めてお互い打ち合う!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> 残り2分間という戦いになりました。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> 残り2分間という戦いになりました。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のジャブ</s> 残り2分間という戦いになりました。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか。</s> </s> </s> </s> 

In [5]:
input = tokenizer.encode("右のジャブ", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のジャブ</s> このストレートどうでしょうか?いいジャブだと思います。右のストレート!右のストレート!今フック入りましたよね</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> このストレートどうでしょうか?いいジャブだと思います。右のストレート!右のストレート顔面をとらえた強いパンチ。右が非常に伸びている。右のストレートです。</s> </s> </s> </s> </s>
右のジャブ</s> このストレートどうでしょうか?いいジャブだと思います。右のストレート<unk> も返して行きます。ボディいいですね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> このストレートどうでしょうか?いいジャブだと思います。右のストレート!右のストレート顔面をとらえた強いパンチ。右が非常に伸びている。右のストレートです。</s> </s> </s> </s> </s>
右のジャブ</s> このストレートどうでしょうか? はいそれとかよりも今少し後ろに体重が移動しました。左のストレート!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> このストレートどうでしょうか?いいジャブだと思います。かなりこのラウンド当たってますよね。右のストレート!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のジャブ</s> このストレートどうでしょうか?いいジャブだと思います。右のストレート!右のストレート顔面をとらえた強いパンチ。右が非常に伸びている。右のストレートです。</s> </s> </s> </s> </s>
右のジャブ</s> 対する<unk> です。いいジャブが出ていますね。お互いいいジャブだと思います。右のストレート! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のジャブ</s> このストレートどうでし

In [6]:
input = tokenizer.encode("左のジャブからワンツー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </s> </s> </s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
左のジャブからワンツー</s> 右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>


In [7]:
input = tokenizer.encode("左のアッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


左のアッパー</s> いま、第二ラウンドのゴングが鳴りました。 この辺りがやっぱり<unk> 本人が話していた、戦うたびに自分の動きとか可能性を感じてしまうというところなんでしょうね。</s>
左のアッパー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。 前の手が重要になってくると思うので、<unk> 選手はサウスポー同士の戦いどのように見ていますか。</s>
左のアッパー</s> <unk> が返す! この距離で左! <unk> の右のアッパー! ここはガードを固めた<unk>! 左のストレート!</s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。。。 右のストレート右のアッパー! ここはガードを固めています<unk> です。</s> </s> </s>
左のアッパー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。 右のストレート! これより第4ラウンド、試合は中盤に移っていきます。</s> </s> </s> </s> </s> </s>
左のアッパー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。 この辺り、序盤の立ち上がりですが、<unk> さんはサウスポー同士の戦いどのように見ていますか。</s>
左のアッパー</s> いま、第二ラウンドのゴングが鳴りました。 持ち味は非常に強いパンチが印象的なこの長井香織です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。 この辺り、序盤の立ち上がりですが、<unk> さんはサウスポー同士の戦いどのように見ていますか。</s>
左のアッパー</s> ここはガードを固めたほうがいいですね。ガードを固めたらジャブどんどん出して。ジャブでシャットアウトした方がいいです。</s> </s> </s> </s> </s> </s> </s> </s>
左のアッパー</s> <unk> が来るところにジャブを入れるだけで止まりはするので。 右のストレート右のアッパー! ここはガードを固め

In [10]:
input = tokenizer.encode("右のアッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


右のアッパー</s> ここはガードを固めたほうがいいですね。ガードを固めたほうがいい相手は<unk> です。 右のアッパー!ガードを固めた<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはガードを固めたほうがいいですね。ガードを固めた方が<unk> はやりやすくなります。 返して行きます<unk> です。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> 少しクラっときたか。足の位置も悪くなっているので。 そうですね。足の位置も悪くなっているので。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> 少しリズムがよくなってきたか。左のストレート! 残り2分間、ポイントになる有効打ををどんどん出せるかどうか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> 少しパンチが当たりましたでしょうか<unk> 。 連打を浴びるシーンもありますが、少しガードが下がってきたでしょうか<unk> 。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> 少しクラっときたか。<unk> の右。 <unk> の右。 ガードの上からでもしっかり手数を出して。最後は気持ちの勝負だ。</s> </s> </s> </s> </s> </s> </s> </s> </s>
右のアッパー</s> ここはガードを固めた左のフック!右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
右のア

In [12]:
input = tokenizer.encode("アッパー", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


アッパー</s> ました。<unk> にも輝いた選手です。 今のこの経歴紹介を聞いていても、<unk> 選手のここまでの戦歴、長さを感じますし、逆に<unk> 選手のシンプルな強さというのは感じますよね。</s>
アッパー</s> これが4戦目。キャリア49戦目。ベテランと若手! <unk> 選手はこのペースで全然いいと思いますね。どんどん前に出る、<unk> 選手。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> これが4戦目。これまでデビューしてから負けを知りません。<unk> 、今リングインです。 地元の観客からは、大きな拍手が送られています。<unk> これが4戦目。</s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> この6ラウンドの戦い方なんですが、<unk> さんは、どういうふうに考えますか? 王者決定戦で、1/3が回ったっていう風になりますか?</s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> <unk> 得意の右アッパー! 今の右のアッパー! あれからどういう風に変化しているのか、すごく楽しみです。 <unk> 得意の右アッパー!</s> </s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> この6ラウンドの戦い方なんですが、<unk> さんは、どういうふうに考えますか? 王者決定戦で、1/3が回ったっていう風になりますか?</s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> いま、第二ラウンドのゴングが鳴りました。 この左のジャブ、アッパー! 持ち味は非常に強いパンチが印象的なこの長井香織です。</s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> この6ラウンドの戦い方なんですが、<unk> さんは、どういうふうに考えますか? 王者決定戦で、1/3が回ったっていう風になりますか?</s> </s> </s> </s> </s> </s> </s> </s>
アッパー</s> この6ラウンドの戦い方なんですが、<unk> さんは、どう

In [22]:
input = tokenizer.encode("カウンター", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


カウンター</s> <unk> が<unk> を破っています。 第7ラウンドを終えて、非常に見応えのある試合が続いています。 ただ、第2ラウンドに<unk> は一つダウンを貰ってるんですよね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> <unk> が後半に入るにつれて、自分のペースに持って行こうとする<unk> です。 第4ラウンド終了です。 <unk> のパンチをもらっているシーンもありましたが、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> <unk> さん、ボディがいいんで、ボディからのフックの返しが、<unk> さんの素早いスピードの打ち終わりを狙ってるイメージで、いいじゃないですか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> <unk> が<unk> を破っています。 第7ラウンドに入っています。 左のストレート! <unk> 選手は中盤にかけて<unk> 勝利、このコロナ禍での応援に来てくださる方に対して、面白い試合をしたいと、言っていますが。</s>
カウンター</s> <unk> さん、ボディがいいんで、ボディからのフックの返しが、<unk> さんの素早いスピードの打ち終わりを狙ってるイメージで、いいじゃないですか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> <unk> さん、ボディがいいんで、ボディからのフックの返しが、<unk> さんの素早いスピードの打ち終わりを狙ってるイメージで、いいじゃないですか。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
カウンター</s> この辺りがやっぱり<unk> 本人が話していた、戦うたびに

In [23]:
input = tokenizer.encode("試合展開", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


試合展開</s> 左のフック!右のストレート右のフック!右のストレート<unk> も返していく! 試合展開は右のストレートから左のフック!右のストレート<unk> も返していく!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> 左のフック!右のストレート右のフック!右のストレート<unk> も返していく! 試合は残り6分。どんどん前に出るの<unk> 。右のストレート!近い距離になってしっかり手数を出して。最後は気持ちの勝負です。</s>
試合展開</s> 左のフック!右のストレート右のフック!右のストレート<unk> も返していく! 試合展開は左のストレート!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> 左のフック!右のストレート右のフック!右のストレート<unk> も返していく! 試合展開は右のストレートから左のフック!右のストレート<unk> も返していく!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> 右のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 第3ラウンド前半が終わりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> 左のフック!右のストレート右のフック!右のストレート<unk> も返していく! 試合展開は右のストレート!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開</s> 左のフック!右のストレート右のフック!右のストレート<unk> も返していく! 試合展開は右のストレートから

In [39]:
input = tokenizer.encode("試合展開について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


試合展開について</s> 序盤から、パンチを出せるか、気持ちとスタミナでカバーできるだけの自信があると話していました。ただ、序盤からどんどん全力で行っても、自分の強みはスタミナだと。</s> </s> </s> </s> </s> </s>
試合展開について</s> 左のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 第3ラウンド前半が終わりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 左のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 第3ラウンド、ここまで序盤は<unk> が落ち着いて、ただ強気のボクシングを見せている。</s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 左のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 第3ラウンド前半が終わりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 左のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 第3ラウンド前半が終わりました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
試合展開について</s> 左のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 序盤の話にもありましたが、右の前手の使い方ですが、<unk> はいいジャブが出ていますね。</s> </s> </s> </s>
試合展開について</s> 左のジャブを打っていって、どんどん自分のペースに持って行こうとする<unk> です。 第3ラウンド、ダウンを食らった直後のラウンドは、ちょっと今までの戦い方に疑問を抱きながらという風に見え

In [32]:
input = tokenizer.encode("ボクシング", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボクシング</s> <unk> 対<unk> の一戦。 2017年の3月にプロテストを合格して順調に勝利を掴めコロナの影響でまた私よりなくならなかったそしてプロ3戦目で<unk> との一戦になりました</s> </s>
ボクシング</s> 左のジャブ。左のフック!右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s>
ボクシング</s> この6ラウンドの戦い方なんですが、<unk> さんは、どういうふうに考えますか? 王者決定戦で、1/3が回ったっていう風になりますか?</s> </s> </s> </s> </s> </s> </s>
ボクシング</s> <unk> 選手がスタミナが切れてきたのか、体が流れ始めてきているので、そうなってくると、<unk> 選手のペースになっていきそうな気がしますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> <unk> 選手がスタミナが切れてきたのか、体が流れ始めてきているので、そうなってくると、<unk> 選手のペースになっていきそうな気がしますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> これは新世代だ。リング上、お互いがお互いをいなす。 第4ラウンド終了です。 <unk> 、ダウンをもらいました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 56キロ契約8回戦で行われます、<unk> 対<unk> の一戦です。 左のジャブを打っていったのは、赤のグローブ<unk> 、<unk> ジム所属です。</s> </s> </s> </s> </s>
ボクシング</s> <unk> 選手がスタミナが切れてきたのか、体が流れ始めてきているので、そうなってくると、<unk> 選手のペースになっていきそうな気がしますね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシング</s> 少しパンチを貰ったか<unk> が終わりにしようか。 なるほど。 <unk

In [29]:
input = tokenizer.encode("ボクシングの試合について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ボクシングの試合について</s> 左のジャブ。 そして右のストレート右のフック! 試合時間が短いという中で、手数もどんどん出していきたい、両者の戦い。</s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> 左のジャブ。そして右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </s>
ボクシングの試合について</s> 左のジャブ。そして右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </s>
ボクシングの試合について</s> 左のジャブ。 そして右のストレート右のフック! 試合全体を通して改めてこの<unk> か選手のボクシングはどう映りましたか?</s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> 左のジャブ。 そして右のストレート右のフック! 試合時間が短いという中で、手数もどんどん出していきたい、</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> 左のジャブ。そして右のストレート右のフック! これ<unk> 選手の右ですね。 距離が近い!右!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
ボクシングの試合について</s> 左のジャブ。そして右のストレート右のフック!右のストレート<unk> も返していく! 試合全体を通して改めてこの<unk> か選手のボクシングはどう映りましたか?</s>
ボクシングの試合について</s> 左のジャブ。そして右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </s>
ボクシングの試合について</s> 左のジャブ。そして右のストレート右のフック!右のストレート<unk> も返していく! こういうところ気持ちのぶつかり合い、しっかりパンチも出ていますね。</s> </

In [30]:
input = tokenizer.encode("今回のボクシングの試合について", return_tensors="pt").to(device)
output = model.generate(input, do_sample=True, top_k=40, top_p=0.95, temperature=0.7, min_length=32, max_length=256, num_return_sequences=10)
# result = tokenizer.batch_decode(output, skip_special_tokens=True)
result = tokenizer.batch_decode(output)
for ret in result:
  print(ret)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


今回のボクシングの試合について</s> <unk> さんは試合開始からして手数も出ている中で、<unk> さんのパンチが当たるというのもありますし、そういうのも狙ってっていますか?</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> <unk> さんは中盤にかけて自分のペースをクシングを貫いていきました。 そのあたり、戦略というのはどうでしょうか? 自分のペースに持って行こうと、自分のスタイル崩してもやったので、このまま行けたらいいかなとは思いますね。</s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> <unk> 選手は判定に移りますが、どうですか、これ判定に移りますが。 少し<unk> のパンチをもらっているシーンもありましたが、大盛りに対して<unk> はちょっとパンチが当たりましたね。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> <unk> さんは試合開始からして手数も出ている中で、<unk> さんのパンチをもらっているシーンもありましたね。 少し<unk> さんのパンチが当たるというところも出ました。</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
今回のボクシングの試合について</s> <unk> 選手は中盤にかけて自分のペースをクシングを貫いていきました。 右のジャブ!そして左のフック!</s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s